# Install Dependencies

In [1]:
# !pip install nltk
# !pip install pandas
# !pip install vaderSentiment

# Run

In [2]:
import json
import re
import nltk
import pandas as pd

# nltk.download('wordnet')
# nltk.download('vader_lexicon')
# nltk.download('punkt')

from nltk.tokenize import RegexpTokenizer, sent_tokenize
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as SIA

## Input a text

In [3]:
text = input()

Bought another 1000 shares. PLTR to the moon 🚀🚀🚀


## Load tickers

In [4]:
df = pd.read_csv('./cleaned_tickers.csv')
tickers = df['ticker'].tolist()
emojis = {
    'rocket': 1.0,
    'gem': 1.0,
}

## Split sentences

In [5]:
text = re.sub(r'\sand\s|\sor\s', '. ', text)
tokenized_str = sent_tokenize(text)

## Apply a sentiment analyzer

In [6]:
sia = SIA()
sia.lexicon.update(emojis)
result = dict()

for sentence in tokenized_str:
    pol_score = sia.polarity_scores(sentence)
    tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|http\S+')
    words = tokenizer.tokenize(sentence)
    ticker = None
    
    for word in words:
        if word in tickers:
            ticker = word

    if not ticker:
        continue
    
    if ticker in result:
        result[ticker] = pol_score['compound'] if pol_score['compound'] > result[ticker] else result[ticker]
    else:
        result[ticker] = pol_score['compound']
    
data = []
for ticker, sentiment_score in result.items():
    data.append({
        'ticker': ticker,
        'sentiment_score': sentiment_score
    })
print(data)

[{'ticker': 'PLTR', 'sentiment_score': 0.6124}]
